# twitter sentiment analysis

In [1]:
import pandas as pd
import  numpy  as  np
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer
import csv
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

### open data

In [9]:
i=0
#recuperation du fichiers csv
df = pd.read_csv("test.csv", encoding='utf-8' )

#nombre de ligne
NumberOfLine = 0
with open('test.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    for line in csv_reader:
        NumberOfLine += 1
        
#taille de chaque tweet
df['pre_clean_len'] = [len(t) for t in df.tweet]
df[:10]
print(NumberOfLine)

ImportError: C extension: 'iNaT' not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace --force' to build the C extensions first.

## data cleaning

### information text

In [45]:
#information sur caque colonne
from pprint import pprint
data_dict = {
    'sentiment':{
        'type':df.label.dtype,
        'description':'sentiment class - 1:negative, 0:positive'
    },
    'text':{
        'type':df.tweet.dtype,
        'description':'tweet text'
    },
    'pre_clean_len':{
        'type':df.pre_clean_len.dtype,
        'description':'Length of the tweet before cleaning'
    },
    'dataset_shape':df.shape
}
pprint(data_dict)

AttributeError: 'DataFrame' object has no attribute 'label'

### Data_cleaner function

In [46]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')
#fonction de nettoyage des tweets
def data_cleaner(tweet):
    soup = BeautifulSoup(tweet, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

### test

In [47]:
test=[]
testing = df.tweet[:100]
test_result = []
for t in testing:
    test_result.append(data_cleaner(t))
print(test_result)

['studiolife aislife requires passion dedication willpower to find newmaterials', 'white supremacists want everyone to see the new birds movie and here why', 'safe ways to heal your acne altwaystoheal healthy healing', 'is the hp and the cursed child book up for reservations already if yes where if no when harrypotter pottermore favorite', 'rd bihday to my amazing hilarious nephew eli ahmir uncle dave loves you and misses', 'choose to be momtips', 'something inside me dies eyes ness smokeyeyes tired lonely sof grunge', 'finished tattoo inked ink loveit thanks aleeee', 'will never understand why my dad left me when was so young deep inthefeels', 'delicious food lovelife capetown mannaepicure resturant', 'dayswasted narcosis infinite ep make me aware grinding neuro bass lifestyle', 'one of the world greatest spoing events lemans teamaudi', 'half way through the website now and allgoingwell very', 'good food good life enjoy and this is called garlic bread iloveit', 'll stand behind this g

### nettoyage des tweets

In [3]:
#taille du csv
nums = [0,NumberOfLine]
print ("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []

#nettoyage des tweet avec pourcentage
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print ("Tweets",i," of ",nums[1]," has been processed")
        i=i+1                                                                  
    clean_tweet_texts.append(data_cleaner(df['tweet'][i]))
clean_tweet_texts
print("Tweets",i," of ",nums[1]," has been processed")
print("csv clean")

#création du tableau de data
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['sentiment'] = df.label

NameError: name 'NumberOfLine' is not defined

In [50]:
#enregistrement du tweet 
clean_df.to_csv('clean_test.csv',encoding='utf-8')
print("csv enregistrer")

csv enregistrer


## Machine learning

###  open data

In [5]:
import pandas as pd
df=pd.read_csv('clean_tweet.csv', sep=',',header=None)
dftest=pd.read_csv('clean_test.csv', sep=',',header=None)

import numpy as np
data=np.array(df)
datatest=np.array(dftest)
print(data)
print(datatest)

[[nan 'text' 'sentiment']
 [0.0
  'when father is dysfunctional and is so selfish he drags his kids into his dysfunction run'
  '0']
 [1.0
  'thanks for lyft credit can not use cause they do not offer wheelchair vans in pdx disapointed getthanked'
  '0']
 ...
 [31959.0 'listening to sad songs on monday morning otw to work is sad'
  '0']
 [31960.0 'sikh temple vandalised in in calgary wso condemns act' '1']
 [31961.0 'thank you for you follow' '0']]
[[nan 'text']
 [0.0
  'studiolife aislife requires passion dedication willpower to find newmaterials']
 [1.0
  'white supremacists want everyone to see the new birds movie and here why']
 ...
 [17194.0
  'hillary campaigned today in ohio omg used words like assets liability never once did clinton say thee word radicalization']
 [17195.0
  'happy at work conference right mindset leads to culture of development organizations work mindset']
 [17196.0 'my song so glad free download shoegaze newmusic newsong']]


In [6]:
#train
print(data)
x=[]
y=[]
for i in range(1,len(data)):
    x.append(data[i][1])
    y.append(data[i][2])

[[nan 'text' 'sentiment']
 [0.0
  'when father is dysfunctional and is so selfish he drags his kids into his dysfunction run'
  '0']
 [1.0
  'thanks for lyft credit can not use cause they do not offer wheelchair vans in pdx disapointed getthanked'
  '0']
 ...
 [31959.0 'listening to sad songs on monday morning otw to work is sad'
  '0']
 [31960.0 'sikh temple vandalised in in calgary wso condemns act' '1']
 [31961.0 'thank you for you follow' '0']]


In [6]:
#test
print(x)
x1=[]

for i in range(1,len(data1)):
    x1.append(data1[i][1])
  

['when father is dysfunctional and is so selfish he drags his kids into his dysfunction run', 'thanks for lyft credit can not use cause they do not offer wheelchair vans in pdx disapointed getthanked', 'bihday your majesty', 'model love take with all the time in ur', 'factsguide society now motivation', 'huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo', 'camping tomorrow danny', 'the next school year is the year for exams can not think about that school exams hate imagine actorslife revolutionschool girl', 'we won love the land allin cavs champions cleveland clevelandcavaliers', 'welcome here it so gr', 'ireland consumer price index mom climbed from previous to in may blog silver gold forex', 'we are so selfish orlando standwithorlando pulseshooting orlandoshooting biggerproblems selfish heabreaking values love', 'get to see my daddy today days gettingfed', 'cnn calls michigan middle school build the wall chant tcot', 'no commen

### transformation of sentences into groups of words

In [7]:
#train
sentences = []
for i in range(len(x)):
    try : 
        sentences.append(x[i].split(" "))
    except :
        print("Erreur no :"+str(i))
        del y[i]
        
for i in range(len(sentences)):
    sentences[i]=[w for w in sentences[i] if w!=""]
    
for i in range(len(sentences)):
    for j in range(len(sentences[i])) :
        sentences[i][j]=sentences[i][j].lower()
sentences
        

Erreur no :5235
Erreur no :30270


[['when',
  'father',
  'is',
  'dysfunctional',
  'and',
  'is',
  'so',
  'selfish',
  'he',
  'drags',
  'his',
  'kids',
  'into',
  'his',
  'dysfunction',
  'run'],
 ['thanks',
  'for',
  'lyft',
  'credit',
  'can',
  'not',
  'use',
  'cause',
  'they',
  'do',
  'not',
  'offer',
  'wheelchair',
  'vans',
  'in',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'your', 'majesty'],
 ['model', 'love', 'take', 'with', 'all', 'the', 'time', 'in', 'ur'],
 ['factsguide', 'society', 'now', 'motivation'],
 ['huge',
  'fan',
  'fare',
  'and',
  'big',
  'talking',
  'before',
  'they',
  'leave',
  'chaos',
  'and',
  'pay',
  'disputes',
  'when',
  'they',
  'get',
  'there',
  'allshowandnogo'],
 ['camping', 'tomorrow', 'danny'],
 ['the',
  'next',
  'school',
  'year',
  'is',
  'the',
  'year',
  'for',
  'exams',
  'can',
  'not',
  'think',
  'about',
  'that',
  'school',
  'exams',
  'hate',
  'imagine',
  'actorslife',
  'revolutionschool',
  'girl'],
 ['we',
  'won',


In [41]:
#test
sentences_test = []
for i in range(len(x1)):
    try : 
        sentences_test.append(x1[i].split(" "))
    except :
        print("Erreur no :"+str(i))
        
for i in range(len(sentences_test)):
    sentences_test[i]=[w for w in sentences_test[i] if w!=""]
    
for i in range(len(sentences_test)):
    for j in range(len(sentences_test[i])) :
        try :
            sentences_test[i][j]=sentences_test[i][j].lower()
        except :
            print("Erreur no :"+str(sentences_test[i][j]))

Erreur no :5306
Erreur no :5451
Erreur no :13356
Erreur no :13404
Erreur no :13841


In [42]:
sentences_test

[['studiolife',
  'aislife',
  'requires',
  'passion',
  'dedication',
  'willpower',
  'to',
  'find',
  'newmaterials'],
 ['white',
  'supremacists',
  'want',
  'everyone',
  'to',
  'see',
  'the',
  'new',
  'birds',
  'movie',
  'and',
  'here',
  'why'],
 ['safe',
  'ways',
  'to',
  'heal',
  'your',
  'acne',
  'altwaystoheal',
  'healthy',
  'healing'],
 ['is',
  'the',
  'hp',
  'and',
  'the',
  'cursed',
  'child',
  'book',
  'up',
  'for',
  'reservations',
  'already',
  'if',
  'yes',
  'where',
  'if',
  'no',
  'when',
  'harrypotter',
  'pottermore',
  'favorite'],
 ['rd',
  'bihday',
  'to',
  'my',
  'amazing',
  'hilarious',
  'nephew',
  'eli',
  'ahmir',
  'uncle',
  'dave',
  'loves',
  'you',
  'and',
  'misses'],
 ['choose', 'to', 'be', 'momtips'],
 ['something',
  'inside',
  'me',
  'dies',
  'eyes',
  'ness',
  'smokeyeyes',
  'tired',
  'lonely',
  'sof',
  'grunge'],
 ['finished', 'tattoo', 'inked', 'ink', 'loveit', 'thanks', 'aleeee'],
 ['will',
  'ne

### vectorization of words

In [8]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(sentences, size=100, min_count=5, workers=4,sorted_vocab=1)
model.train(sentences,total_examples=len(sentences), epochs=10)



C:\Users\Thomas\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


(2656905, 3724020)

In [9]:
#train
# max_len=50 # Maximum size of a sentence
max_len=0
sentences_v=[]
unusued_word=[]
for s in sentences:
    temp=[]
    for w in s :
        try :
            temp.append(model[w])
        except : 
            unusued_word.append(w)
    sentences_v.append(temp)

for i in range(len(sentences_v)):
    if(len(sentences_v[i])>max_len):
        max_len=len(sentences_v[i])
        
for i in range(len(sentences_v)):
    sentence_length = len(sentences_v[i])
    if(sentence_length<max_len):    
        for j in range(max_len-sentence_length):
            sentences_v[i].append([0]*100)
            
##unusued_word

          

C:\Users\Thomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [43]:
#test
# max_len=50 # Maximum size of a sentence
max_len=30
sentences_v=[]
unusued_word=[]
for s in sentences_test:
    temp=[]
    for w in s :
        try :
            temp.append(model[w])
        except : 
            unusued_word.append(w)
    sentences_v.append(temp)

for i in range(len(sentences_v)):
    if(len(sentences_v[i])>max_len):
        max_len=len(sentences_v[i])
        
for i in range(len(sentences_v)):
    sentence_length = len(sentences_v[i])
    if(sentence_length<max_len):    
        for j in range(max_len-sentence_length):
            sentences_v[i].append([0]*100)
     

C:\Users\Thomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [44]:
x_vec_test = np.array(sentences_v)

In [45]:
x_vec_test.shape

(17192, 30, 100)

In [46]:
len(x_vec_test)

17192

In [10]:
from sklearn.model_selection import train_test_split
x_vec_train,x_vec_test,y_vec_train,y_vec_test = train_test_split(sentences_v,y,test_size=0.1)
x_vec_train=np.array(x_vec_train)
x_vec_test=np.array(x_vec_test)

#from keras.utils import to_categorical
#y_vec_train=to_categorical(y_vec_train)
#y_vec_test=to_categorical(y_vec_test)  

In [11]:

dataset_size = len(x_vec_train)
x_vec_train = x_vec_train.reshape(dataset_size,-1)



dataset_size = len(x_vec_test)
x_vec_test = x_vec_test.reshape(dataset_size,-1)


## Naive Bayes classifier

### création modèle NB

In [24]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_vec_train,y_vec_train)

GaussianNB(priors=None)

### resultat NB

In [26]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(x_vec_test)
cf = confusion_matrix(y_vec_test,y_pred)
score = clf.score(x_vec_test,y_vec_test)

In [29]:
print("confusion matrix")
print(cf)
print("score")
print(score)

x_test_test = np.array(sentences_v[:300][:])
dataset_size = len(x_test_test)
x_test_test = x_test_test.reshape(dataset_size,-1)




y_pred = clf.predict(x_test_test)



for i in range(301):
    print("Phrase : ")
    print(sentences[i])
    print("Prédiction : "+str(y_pred[i]))
    print("Label réel : "+str(y[i]))



confusion matrix
[[  71 2898]
 [   2  225]]
score
0.09261576971214018
Phrase : 
['when', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', 'run']
Prédiction : 1
Label réel : 0
Phrase : 
['thanks', 'for', 'lyft', 'credit', 'can', 'not', 'use', 'cause', 'they', 'do', 'not', 'offer', 'wheelchair', 'vans', 'in', 'pdx', 'disapointed', 'getthanked']
Prédiction : 1
Label réel : 0
Phrase : 
['bihday', 'your', 'majesty']
Prédiction : 1
Label réel : 0
Phrase : 
['model', 'love', 'take', 'with', 'all', 'the', 'time', 'in', 'ur']
Prédiction : 1
Label réel : 0
Phrase : 
['factsguide', 'society', 'now', 'motivation']
Prédiction : 1
Label réel : 0
Phrase : 
['huge', 'fan', 'fare', 'and', 'big', 'talking', 'before', 'they', 'leave', 'chaos', 'and', 'pay', 'disputes', 'when', 'they', 'get', 'there', 'allshowandnogo']
Prédiction : 1
Label réel : 0
Phrase : 
['camping', 'tomorrow', 'danny']
Prédiction : 1
Label réel : 0
Phrase : 
['

Prédiction : 1
Label réel : 0
Phrase : 
['take', 'out', 'the', 'trash', 'america', 'voted', 'against', 'hate', 'voted', 'against', 'voted', 'against', 'vot']
Prédiction : 1
Label réel : 1
Phrase : 
['fathers', 'day', 'to', 'all', 'the', 'amazing', 'dads', 'out', 'there', 'that', 'give', 'up', 'so', 'much', 'for', 'their', 'families', 'enjoy', 'your']
Prédiction : 1
Label réel : 0
Phrase : 
['does', 'magnettherapy', 'really', 'work', 'altwaystoheal', 'healing', 'healthy']
Prédiction : 1
Label réel : 0
Phrase : 
['sebbo', 'has', 'said', 'it', 'all']
Prédiction : 1
Label réel : 0
Phrase : 
['eg', 'smile', 'smile', 'egsmile', 'torio']
Prédiction : 1
Label réel : 0
Phrase : 
['brilliant', 'service', 'at', 'your', 'kettering', 'branch', 'today', 'customer']
Prédiction : 1
Label réel : 0
Phrase : 
['will', 'be', 'here', 'for', 'screening', 'will', 'miss', 'it', 'so', 'will', 'sing', 'all', 'the', 'songs', 'from', 'potp', 'and', 'shock', 'treatment', 'because', 'can']
Prédiction : 1
Label réel

IndexError: index 300 is out of bounds for axis 0 with size 300

## SVM

### création modèle SVM

In [30]:
from sklearn.svm import SVC
clf_svm = SVC(verbose=True,class_weight="balanced")
clf_svm.fit(x_vec_train,y_vec_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

### resultats SVM

In [31]:
clf_svm.score(x_vec_test,y_vec_test)

from sklearn.metrics import confusion_matrix
y_pred = clf_svm.predict(x_vec_test)
cf = confusion_matrix(y_vec_test,y_pred)


In [33]:
score = clf_svm.score(x_vec_test,y_vec_test)
print("confusion matrix")
print(cf)
print("score")
print(score)

print_sentences =  sentences[300:600][:]
x_test_test = np.array(sentences_v[300:600][:])
dataset_size = len(x_test_test)
x_test_test = x_test_test.reshape(dataset_size,-1)

y_pred = clf_svm.predict(x_test_test)

for i in range(len(print_sentences)):
    print("Phrase : ")
    print(print_sentences[i])
    print("Prédiction : "+str(y_pred[i]))
    print("Label réel : "+str(y[i]))




confusion matrix
[[2524  445]
 [  41  186]]
score
0.8479349186483104
Phrase : 
['stuck', 'in', 'athens', 'instead', 'of', 'santorini', 'because', 'said', 'it', 'was', 'to', 'windy', 'to', 'land', 'shocking', 'service', 'on', 'every', 'level', 'since']
Prédiction : 0
Label réel : 0
Phrase : 
['these', 'happy', 'folks', 'had', 'the', 'very', 'first', 'freakshake', 'at', 'the', 'launch', 'freaks', 'yum', 'dalston']
Prédiction : 0
Label réel : 0
Phrase : 
['what', 'huge', 'crowd', 'for', 'trump']
Prédiction : 1
Label réel : 0
Phrase : 
['am', 'thankful', 'for', 'saturdays', 'thankful', 'positive']
Prédiction : 0
Label réel : 0
Phrase : 
['good', 'god', 'why']
Prédiction : 0
Label réel : 0
Phrase : 
['enjoying', 'these', 'delicious', 'melons', 'vegan', 'organic', 'fresh', 'fruit', 'organic', 'healthy', 'healthyliving', 'life']
Prédiction : 0
Label réel : 0
Phrase : 
['first', 'months', 'then', 'weeks', 'now', 'it', 'only', 'days', 'till', 'the', 'beginning', 'of', 'the', 'end', 'of', 'am', 

## RandomForest

### RandomForest model creation


In [38]:
x_vec_train.shape
from sklearn.ensemble import RandomForestClassifier
score=[]

clf_forest = RandomForestClassifier(n_estimators=45,class_weight="balanced",verbose=1,n_jobs=2)
clf_forest.fit(x_vec_train,y_vec_train)

[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:   18.0s finished


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=45, n_jobs=2, oob_score=False, random_state=None,
            verbose=1, warm_start=False)

### results random forest

In [39]:
clf_forest.score(x_vec_test,y_vec_test)

[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished


0.9493116395494368

In [40]:
clf_forest.score(x_vec_test,y_vec_test)

from sklearn.metrics import confusion_matrix
y_pred = clf_forest.predict(x_vec_test)
cf = confusion_matrix(y_vec_test,y_pred)
score = clf_forest.score(x_vec_test,y_vec_test)
print("confusion matrix")
print(cf)
print("score")
print(score)

[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished


confusion matrix
[[2961    9]
 [ 153   73]]
score
0.9493116395494368


[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished


In [48]:
#test 
print_sentences =  sentences_test[300:600][:]
x_test_test = np.array(sentences_v[300:600][:])
dataset_size = len(x_test_test)
x_test_test = x_test_test.reshape(dataset_size,-1)

y_pred = clf_forest.predict(x_test_test)

for i in range(len(print_sentences)):
    print("Phrase : ")
    print(print_sentences[i])
    print("Prédiction : "+str(y_pred[i]))




Phrase : 
['ccs', 'without', 'aciiii', 'eki', 'liani', 'fetty', 'ccs', 'emergency', 'team', 'friend', 'family', 'like', 'love']
Prédiction : 0
Phrase : 
['black', 'lives', 'matter', 'today', 'day']
Prédiction : 0
Phrase : 
['usd', 'jpy', 'remains', 'weak', 'below', 'blog', 'silver', 'gold', 'forex']
Prédiction : 0
Phrase : 
['character', 'customization', 'confirmed', 'pokemonsunmoon']
Prédiction : 0
Phrase : 
['emma', 'great', 'meeting', 'you', 'yesterday', 'will', 'email', 'as', 'have', 'an', 'oppounity', 'for', 'you', 'already']
Prédiction : 0
Phrase : 
['my', 'poor', 'babes', 'missed', 'water', 'play', 'this', 'morning', 'at', 'school', 'figured', 'they', 'do', 'water', 'play', 'in', 'the', 'afternoon', 'poorbaby']
Prédiction : 0
Phrase : 
['let', 'there', 'be', 'light', 'hmmm', 'prefer', 'spotlight', 'vibrant', 'alive', 'joy', 'sparkle']
Prédiction : 0
Phrase : 
['today', 'was', 'good', 'day', 'love', 'goodday', 'smiles', 'goodangle', 'instasmile', 'instagram']
Prédiction : 0
Phras

In [41]:
score = clf_forest.score(x_vec_test,y_vec_test)
print("confusion matrix")
print(cf)
print("score")
print(score)


print_sentences =  sentences[800:1100][:]
x_test_test = np.array(sentences_v[800:1100][:])
dataset_size = len(x_test_test)
x_test_test = x_test_test.reshape(dataset_size,-1)

y_pred = clf_forest.predict(x_test_test)

for i in range(len(print_sentences)):
    print("Phrase : ")
    print(print_sentences[i])
    print("Prédiction : "+str(y_pred[i]))
    print("Label réel : "+str(y[i]))



[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished


confusion matrix
[[2961    9]
 [ 153   73]]
score
0.9493116395494368


[Parallel(n_jobs=2)]: Done  45 out of  45 | elapsed:    0.0s finished


Phrase : 
['if', 'you', 'want', 'creative', 'workers', 'give', 'them', 'enough', 'time', 'to', 'play', 'success', 'quote']
Prédiction : 0
Label réel : 0
Phrase : 
['my', 'hea', 'prayers', 'are', 'with', 'the', 'orlando', 'victims', 'their', 'families', 'and', 'the', 'lgbt', 'community']
Prédiction : 0
Label réel : 0
Phrase : 
['jan', 'interested', 'in', 'how', 'discrimination', 'and', 'has', 'affected', 'the', 'criminal', 'justice', 'system']
Prédiction : 1
Label réel : 0
Phrase : 
['friends', 'forever', 'friends', 'sweden', 'reunion', 'grateful', 'love', 'egna', 'hem']
Prédiction : 0
Label réel : 0
Phrase : 
['keep', 'up', 'the', 'opposition', 'to', 'endthenation']
Prédiction : 1
Label réel : 0
Phrase : 
['love', 'pizza', 'pizzaslice', 'pizzaslice', 'aoyama', 'pizza', 'yummy', 'delicias', 'goodtimeswithgo']
Prédiction : 0
Label réel : 0
Phrase : 
['my', 'two', 'loves', 'beer', 'craftbeee', 'dog', 'malinoislovers', 'malinois', 'passion', 'friend']
Prédiction : 0
Label réel : 0
Phrase :

Label réel : 0
Phrase : 
['he', 'the', 'only', 'one', 'where', 'the', 'majority', 'of', 'fans', 'like', 'him', 'salty', 'gsw', 'fans', 'just', 'needed', 'someone', 'to', 'be', 'about']
Prédiction : 0
Label réel : 0
Phrase : 
['changing', 'on', 'daily', 'basis', 'weeks', 'until', 'handover', 'network', 'team', 'in', 'soon', 'then', 'av']
Prédiction : 0
Label réel : 0
Phrase : 
['when', 'is', 'this', 'coming', 'to', 'singapore']
Prédiction : 0
Label réel : 1
Phrase : 
['sma', 'baby', 'socks', 'gbp', 'get', 'here', 'shop', 'cool', 'home', 'fun']
Prédiction : 0
Label réel : 0
Phrase : 
['liked', 'video', 'from', 'sjw', 'retard', 'wants', 'more']
Prédiction : 1
Label réel : 0
Phrase : 
['so', 'woh', 'the', 'wait', 'doubledown', 'my', 'wife', 'is', 'veganbeerfest', 'vegan']
Prédiction : 0
Label réel : 0
Phrase : 
['lmfao', 'deadass']
Prédiction : 0
Label réel : 0
Phrase : 
['what', 'an', 'amazing', 'interview', 'bjmckenzie', 'born', 'win', 'bjmuzic']
Prédiction : 0
Label réel : 1
Phrase : 
[

Prédiction : 0
Label réel : 0
Phrase : 
['turning', 'year', 'younger', 'newyear', 'newblessings', 'bihday', 'to', 'me', 'thankful', 'grateful']
Prédiction : 0
Label réel : 0
Phrase : 
['can', 'not', 'wait', 'for', 'wales', 'vs', 'nz', 'tonight', 'at', 'eden', 'park', 'cymruambyth', 'come', 'on']
Prédiction : 0
Label réel : 0
Phrase : 
['this', 'will', 'break', 'my', 'hea', 'it', 'will', 'teamandre', 'idontcareifhesbritish']
Prédiction : 0
Label réel : 0
Phrase : 
['buzzing', 'that', 'this', 'time', 'weeks', 'and', 'will', 'be', 'lying', 'on', 'beach', 'in', 'phuket', 'hurryup']
Prédiction : 0
Label réel : 0
Phrase : 
['happiness', 'realtimechem', 'chemistry', 'education', 'very', 'sma', 'and', 'practical']
Prédiction : 0
Label réel : 0
Phrase : 
['rooftops', 'sunday', 'weekend', 'drinks', 'la', 'night', 'views', 'summer', 'fullmoon', 'dtla', 'fun']
Prédiction : 0
Label réel : 0
Phrase : 
['but', 'how', 'does', 'an', 'yrs', 'old', 'save', 'up', 'enough', 'money', 'to', 'buy', 'motorcycl

### test final

In [23]:
clf_forest.score(x_vec_test,y_vec_test)


0.9515018773466833

In [39]:
import  pandas  as  pd
import  numpy  as  np
import matplotlib.pyplot as plt
import csv
x=1
y=0
z=0
with open('train.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)

    next(csv_reader)
   
    for line in csv_reader:
        x=x+1
        if line[1] == ("neg"):
            y=y+1
        else:
            z=z+1
print("for",x,"tweet there are",y,"racist/sexist &",z,"neutral." )

for 31963 tweet there are 2242 racist/sexist & 29720 neutral.
